In [223]:
import tifffile
import python_test as pt
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import os
import pickle

In [226]:
def filter_masks(masks):
    # REMOVE CELLS THAT ARE NOT PRESENT IN AT LEAST 50% OF THE IMAGES
    labels, counts = pt.get_common_cells(masks, occurrence=50)
    masks[~np.isin(masks, labels)] = 0

    # REMOVE CELLS ON BORDER
    dimensions = masks.shape[1:]
    new_labels = []
    for lbl in labels:
        poss = np.array(np.where(masks == lbl)[1:])
        if 0 in poss or dimensions[0]-1 in poss: # Supposing quadratic images
            masks[masks == lbl] = 0
            continue
        new_labels.append(lbl)

    # REMOVE CELLS WITH TOO DRASTICAL CHANGES IN SIZE
    fltr_param = 0.5      # 0.5 was found to be a good value
    fltd_labels = []
    # print(labels)
    for label in new_labels:
        area_old = np.sum(masks[0] == label)
        bad_label = False
        counter = 0
        for mask in masks[1:]:
            area_new = np.sum(mask==label)
            if area_new == 0:
                continue
            if area_old == 0:
                area_old = area_new
                continue
            if abs((area_new-area_old)/min(area_new,area_old)) > fltr_param:
                counter += 1
                if counter >= 3:
                    bad_label = True
                    masks[masks == label] = 0
                    break
            area_old=area_new
        if not bad_label:
            fltd_labels.append(label)
    return masks, labels, fltd_labels

### Run code on one recording

In [201]:
masks_path = "//storage3.ad.scilifelab.se/alm/BrismarGroup/Hanna/240115_correlation_data/masks_tracked/2APB + OUA_15s.tif"
masks = tifffile.imread(masks_path)

In [206]:
masks_fltd, labels, fltd_labels = filter_masks(masks)

KeyboardInterrupt: 

In [203]:
# These cells are removed
print(set(labels) - set(fltd_labels))

{1, 3, 4, 7, 8, 9, 11, 14, 17, 18, 20, 24, 26, 27, 28, 30, 34, 35, 36, 37, 39, 40, 41, 42, 43, 49, 51, 52, 180, 54, 55, 57, 59, 62, 65, 67, 69, 71, 72, 73, 74, 75, 76, 79, 84, 87, 88, 91, 92, 93, 96, 97, 98, 100, 103, 104, 106, 108, 110, 113, 124, 126}


In [205]:
# These cells are kept
print(set(fltd_labels))

{12, 21, 22, 25, 32, 38, 44, 46, 53, 56, 60, 61, 64, 66, 70, 77, 78, 81, 82, 86, 89, 90, 94, 99, 101, 102, 125}


### Run code on all recordings and generate pkl files

In [225]:
def create_folder(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully.")
    else:
        print(f"Folder '{folder_path}' already exists.")

In [229]:
pkl_path = "//storage3.ad.scilifelab.se/alm/BrismarGroup/Hanna/Master2023/data/pkl/"
masks_path = "//storage3.ad.scilifelab.se/alm/BrismarGroup/Hanna/240115_correlation_data/masks_tracked/"
save_path = "//storage3.ad.scilifelab.se/alm/BrismarGroup/Hanna/Master2023/data/filtered_pkl/"

all_masks = os.listdir(masks_path)
create_folder(save_path)

for f in all_masks:
    new_dict = {}
    with open(masks_path+f, 'rb') as x:
        masks = tifffile.imread(x)
    masks_fltd, labels, fltd_labels = filter_masks(masks)

    with open(pkl_path+f[:-4]+".pkl", 'rb') as x:
        d = pd.read_pickle(x)

    for lbl in fltd_labels:
        new_dict[lbl] = d[lbl]

    j = open(save_path+f[:-4]+".pkl","wb")
    pickle.dump(new_dict,j)
    j.close()



Folder '//storage3.ad.scilifelab.se/alm/BrismarGroup/Hanna/Master2023/data/filtered_pkl/' created successfully.
